In [1]:
%matplotlib inline
#Reference from https://pytorch.org/

In [2]:
from __future__ import print_function, division

import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.Resize([224, 224]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(180),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize([224, 224]),
        #transforms.Resize(256),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../AddMoreData061823/Class3/TTV'
l = []
l.append(datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']))
l.append(datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']))
l.append(datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']))
l.append(datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']))
#l.append(datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test']))
image_datasetss = torch.utils.data.ConcatDataset(l)
image_datasets = {}
image_datasets['train'] = image_datasetss
image_datasets['test'] = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])
image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['test'])
testfileNames = [x[0] for x in image_datasets['test'].imgs]



dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=False, num_workers=4)
              for x in ['test']}


#dataloaders['testfileNames'] = torch.utils.data.DataLoader(testfileNames, batch_size=5,
                                             #shuffle=False, num_workers=4)

print(dataloaders)

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test', 'val']}
class_names = image_datasets['test'].classes
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(len( image_datasets['train']))
print(len( image_datasets['val']))
print(len( image_datasets['test']))


{'test': <torch.utils.data.dataloader.DataLoader object at 0x000001C8ED712E20>}
['0', '1', '2']
3016
93
98


In [10]:
#model_ft = models.resnet50(pretrained=True) #load googlenet.
#model_ft = models.googlenet(pretrained=True) #load googlenet.
#num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 3)

#model_ft = models.vgg16(pretrained=True) #load googlenet.
#model_ft.classifier[-1] = nn.Linear(4096, 3)

model_ft = models.convnext_base(pretrained=True) #load googlenet.
model_ft.classifier[-1] = nn.Linear(1024, 10)


# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).


model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

ep = 100
#PATH = './trained models/ResNet-TTVClass3-BestEP%s.pth' %(ep)

PATH = './trained models/convnext_base-TTVClass3-BestEP100.pth'
model_ft.load_state_dict(torch.load(PATH))
model_ft.eval()

total_params = sum(p.numel() for p in model_ft.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 87576714


In [11]:
correct = 0
total = 0
nb_classes = 3
confusion_matrix = torch.zeros(nb_classes, nb_classes)

results = {}
results['name'], results['act.'], results['pred.'] = [], [], []
oo = 0
with torch.no_grad():
    for data in dataloaders['test']:
        #print(data)
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = model_ft(images)
        predicted = torch.round(outputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                #print(labels, predicted )
                #print(t.long(), p.long())
                confusion_matrix[t.long(), p.long()] += 1
                results['name'].append(testfileNames[oo])
                results['act.'].append(torch.Tensor.cpu(t.long()))
                results['pred.'].append(torch.Tensor.cpu(p.long()))
                oo+=1
        #break
        

print('Accuracy of the network on the %s test images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)
#print(confusion_matrix/(total/2)) #Normalizing
b= torch.sum(confusion_matrix, dim=1)
print(confusion_matrix/torch.reshape(b, (-1, 1))) #Normalizing

tr = pd.DataFrame (results)
tr.to_excel('results.xlsx', index=False)

Accuracy of the network on the 98 test images: 82 %
tensor([[ 9.,  5.,  0.],
        [ 3., 63.,  3.],
        [ 1.,  5.,  9.]])
tensor([[0.6429, 0.3571, 0.0000],
        [0.0435, 0.9130, 0.0435],
        [0.0667, 0.3333, 0.6000]])


In [6]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            m = nn.Softmax(dim=1)
            proOutput = m(outputs)
            pros = proOutput.detach().cpu().numpy()
            #print(pros)
            #_, preds = torch.max(outputs, 1)
            #pro = torch.max(proOutput, dim=1)[0]
            #_, predsmin = torch.min(outputs, 1)
            #promin = torch.min(proOutput, dim=1)[0]

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                #ax.set_title('predicted: %s with %s probabilities' %(class_names[preds[j]], round(pro[j].item(),4)))
                print('Actual label: %s' %(class_names[labels[j].item()]))
                for jj in range(3):
                    print('predicted: %s with %s probabilities' %(class_names[jj], round(pros[j][jj],4)))
                #print('predicted: %s with %s probabilities' %(class_names[1], round(pros[j][1],4)))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
                
        model.train(mode=was_training)

#visualize_model(model_ft)

visualize_model(model_ft, 36)

Actual label: 0
predicted: 0 with 1.0 probabilities
predicted: 1 with 0.0 probabilities
predicted: 2 with 0.0 probabilities


Actual label: 0
predicted: 0 with 1.0 probabilities
predicted: 1 with 0.0 probabilities
predicted: 2 with 0.0 probabilities


Actual label: 0
predicted: 0 with 0.3405 probabilities
predicted: 1 with 0.6595 probabilities
predicted: 2 with 0.0 probabilities


Actual label: 0
predicted: 0 with 0.9887 probabilities
predicted: 1 with 0.0113 probabilities
predicted: 2 with 0.0 probabilities


Actual label: 0
predicted: 0 with 0.0004 probabilities
predicted: 1 with 0.9914 probabilities
predicted: 2 with 0.0083 probabilities


...

Actual label: 1
predicted: 0 with 0.0813 probabilities
predicted: 1 with 0.8267 probabilities
predicted: 2 with 0.092 probabilities


Actual label: 1
predicted: 0 with 0.0004 probabilities
predicted: 1 with 0.974 probabilities
predicted: 2 with 0.0256 probabilities


Actual label: 1
predicted: 0 with 0.0 probabilities
predicted: 1 with 1.0 probabilities
predicted: 2 with 0.0 probabilities


Actual label: 1
predicted: 0 with 0.0 probabilities
predicted: 1 with 0.9999 probabilities
predicted: 2 with 1e-04 probabilities


Actual label: 1
predicted: 0 with 0.0 probabilities
predicted: 1 with 0.9999 probabilities
predicted: 2 with 1e-04 probabilities
